In [221]:
import numpy as np
import scipy

In [222]:
network = 'data/I210'

In [223]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [224]:
# in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])

# in the case where the index of the od pairs does not begin by 0, we rename the od pairs
first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
graph[:,1] = graph[:,1]-first_index_od
graph[:,2] = graph[:,2]-first_index_od
demand[:,0] = demand[:,0] - first_index_od
demand[:,1] = demand[:,1] - first_index_od

In [225]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])

In [228]:
demand[0][2] = 10000
print(demand)

[[   18.    19. 10000.]]


In [229]:
def travel_time(f):
    return graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)

In [230]:
def hessian(f):
    return graph[:,4] + 2*graph[:,5]*(f) + 3*graph[:,6]*(f**2) + 4*graph[:,7]*(f**3)

In [231]:
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
print(nb_nodes)
print(nb_links)

20
41


In [239]:
debug_f = False
debug_t = True
h = np.zeros(paths_matrix.shape[0])
h[0] = demand[0][2]

free_flow = np.zeros(nb_links)
if debug_f:
    print(h)
    print(travel_time(free_flow))

flow_from_h = paths_matrix.T @  h


## Compute the travel time of every paths
if debug_t:
    print(flow_from_h)
    print(travel_time(flow_from_h))
hessian_h = paths_matrix @ np.diag(hessian(flow_from_h)) @ paths_matrix.T
grad_h = paths_matrix @ travel_time(flow_from_h)
h_update = - np.linalg.pinv(paths_matrix @ np.diag(hessian(flow_from_h)) @ paths_matrix.T) @ grad_h

# now you have to change h_update such that sum(h_update) = 0 and h(k+1) >= 0
# let do that sum(h_update) = 0
dhp = sum(h_update) / h_update.shape
h_update = h_update - dhp
print("#########")
print(sum(h_update))

print(h_update)
if debug_f:
    print(travel_time(flow_from_h))
    print(travel_time(flow_from_h) * paths_matrix)
# print((paths_matrix @ hessian(flow_from_h) @ paths_matrix.T).shape)
if debug_f:
    print(hessian(flow_from_h))
    print(paths_matrix @ hessian(flow_from_h))
#delta = 
#print(travel_time())


[10000.     0. 10000.     0. 10000.     0. 10000.     0. 10000.     0.
 10000. 10000.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0. 10000.
 10000.]
[3.3120e+16 1.1500e+00 3.3120e+16 1.1400e+00 4.0320e+16 1.1300e+00
 3.0240e+16 1.0500e+00 5.4000e+16 1.0400e+00 2.6160e+16 2.7600e+16
 4.8000e-01 3.6000e-01 1.1400e+00 4.8000e-01 3.9000e-01 1.1300e+00
 5.8000e-01 3.8000e-01 1.0500e+00 4.3000e-01 4.8000e-01 1.0400e+00
 7.7000e-01 5.0000e-01 1.0900e+00 4.7000e-01 3.6000e-01 1.3800e+00
 3.9000e-01 1.3800e+00 3.8000e-01 1.6800e+00 4.8000e-01 1.2500e+00
 5.0000e-01 2.2200e+00 4.7000e-01 8.4480e+12 1.1136e+13]
#########
-3.140598892059643e-12
[-139.02147634  224.22852366  -43.77147634 -147.27147634  -56.77147634
  -29.27147634  -79.77147634  -49.77147634  -63.77147634 -157.77147634
   16.22852366  -49.77147634  -37.52147634  -20.27147634  -34.77147634
  

In [77]:
print(demand)

[[1.8e+01 1.9e+01 2.5e+04]]


In [78]:
# graph_dict[3] = {0: 5}
print(graph_dict)

{0: {1: 0, 6: 1}, 1: {2: 2, 7: 3}, 2: {3: 4, 8: 5}, 3: {4: 6, 9: 7}, 4: {5: 8, 10: 9}, 5: {11: 10}, 6: {0: 11, 7: 12, 12: 13}, 7: {1: 14, 8: 15, 13: 16}, 8: {2: 17, 9: 18, 14: 19}, 9: {3: 20, 10: 21, 15: 22}, 10: {4: 23, 11: 24, 16: 25}, 11: {5: 26, 17: 27, 19: 39}, 12: {6: 28, 13: 29}, 13: {7: 30, 14: 31}, 14: {8: 32, 15: 33}, 15: {9: 34, 16: 35}, 16: {10: 36, 17: 37}, 17: {11: 38}, 18: {6: 40}}


In [79]:
## DO THE CASE WHERE THERE IS A LOOP IN THE NETWORK
## USE A CLASS, YOU CANNOT USE LOCAL SET IN PYTHON

def add_node(o, d, test, o_tmp, tab):
    # print("If I want to access o=" + str(o_tmp) + " from d=" + str(d) + ", i can go through n="+ str(o))
    if test[o][d]==-1:
        test[o][d] = set()
    test[o][d].add(o_tmp)
    # print("I've just put test[" + str(o_tmp) + "][" + str(d) + "]=" + str(o))
    print(tab + "test[" + str(o) + "][" + str(d) + "]=" + str(test[o][d]))
    # print()

def update(o, d, test, pred, visited, tab):
    # print("I am in the update corresponding to node " + str(d))
    # print("First, i need to put the predessor of d:" + str(predecessor))
    # add_node(o, d, test, o)
    for o_pred in pred:
        # print("Let do it for o:" + str(o_tmp))
        add_node(o_pred, d, test, o, tab)
    
    predecessor_tmp = pred.copy() # HERE THE COPY IS NOT A REAL COPY! STILL HAVE THE SAME MEMORY
    predecessor_tmp.add(d)
    print(tab + str(d) + "; pred: " + str(pred))
    try:
        # print("Now, let do it again for the node after d=" + str(d))
        # print("The node which follow d are:")
        for d_tmp in graph_dict[d].keys():
            print(tab + str(d) + "->" + str(d_tmp))
            if d_tmp not in visited:
                visited.add(d_tmp)
                update(d, d_tmp, test, predecessor_tmp, visited.copy(), tab + "   " + str(d) + ":   ")
    except:
        return

In [88]:
test = np.array(np.zeros(shape=(nb_nodes, nb_nodes))-1,dtype=object)
for o in demand[:,0]:
    o = int(o)
# for o in graph_dict.keys():
    # print()
    # print("************************************")
    # print("I consider " + str(o))
    predecessor = set()
    predecessor.add(o)
    for d in graph_dict[o].keys():
        print("I will update " + str(d))
        # print("I need to put test[" + str(o) + "][" + str(d) + "]=" + str(o))
        # print("test[" + str(o) + "][" + str(d) + "]=" + str(test[o][d]))
        update(o, d, test, predecessor, predecessor.copy(), "")

I will update 6
test[18][6]={18}
6; pred: {18}
6->0
   6:   test[18][0]={6}
   6:   test[6][0]={6}
   6:   0; pred: {18, 6}
   6:   0->1
   6:      0:   test[0][1]={0}
   6:      0:   test[18][1]={0}
   6:      0:   test[6][1]={0}
   6:      0:   1; pred: {0, 18, 6}
   6:      0:   1->2
   6:      0:      1:   test[0][2]={1}
   6:      0:      1:   test[1][2]={1}
   6:      0:      1:   test[18][2]={1}
   6:      0:      1:   test[6][2]={1}
   6:      0:      1:   2; pred: {0, 1, 18, 6}
   6:      0:      1:   2->3
   6:      0:      1:      2:   test[0][3]={2}
   6:      0:      1:      2:   test[1][3]={2}
   6:      0:      1:      2:   test[2][3]={2}
   6:      0:      1:      2:   test[6][3]={2}
   6:      0:      1:      2:   test[18][3]={2}
   6:      0:      1:      2:   3; pred: {0, 1, 2, 6, 18}
   6:      0:      1:      2:   3->4
   6:      0:      1:      2:      3:   test[0][4]={3}
   6:      0:      1:      2:      3:   test[1][4]={3}
   6:      0:      1:      2:      3: 

   6:      0:      6:      7:      8:      9:      10:      11:   test[11][19]={11}
   6:      0:      6:      7:      8:      9:      10:      11:   19; pred: {0, 18, 6, 7, 8, 9, 10, 11}
   6:      0:      6:      7:      8:      9:   10->16
   6:      0:      6:      7:      8:      9:      10:   test[0][16]={10, 15}
   6:      0:      6:      7:      8:      9:      10:   test[18][16]={10, 15}
   6:      0:      6:      7:      8:      9:      10:   test[6][16]={10, 15}
   6:      0:      6:      7:      8:      9:      10:   test[7][16]={10, 15}
   6:      0:      6:      7:      8:      9:      10:   test[8][16]={10, 15}
   6:      0:      6:      7:      8:      9:      10:   test[9][16]={10, 15}
   6:      0:      6:      7:      8:      9:      10:   test[10][16]={10}
   6:      0:      6:      7:      8:      9:      10:   16; pred: {0, 18, 6, 7, 8, 9, 10}
   6:      0:      6:      7:      8:      9:      10:   16->10
   6:      0:      6:      7:      8:      9:      10:   1

   6:      7:      1:      2:      3:      4:      10:      16:   test[6][17]={16, 11}
   6:      7:      1:      2:      3:      4:      10:      16:   test[7][17]={16, 11}
   6:      7:      1:      2:      3:      4:      10:      16:   test[10][17]={16, 11}
   6:      7:      1:      2:      3:      4:      10:      16:   test[16][17]={16, 11}
   6:      7:      1:      2:      3:      4:      10:      16:   test[18][17]={16, 11}
   6:      7:      1:      2:      3:      4:      10:      16:   17; pred: {1, 2, 3, 4, 6, 7, 10, 16, 18}
   6:      7:      1:      2:      3:      4:      10:      16:   17->11
   6:      7:      1:      2:   3->9
   6:      7:      1:      2:      3:   test[1][9]={8, 3, 15}
   6:      7:      1:      2:      3:   test[2][9]={8, 3}
   6:      7:      1:      2:      3:   test[18][9]={8, 3, 15}
   6:      7:      1:      2:      3:   test[3][9]={3}
   6:      7:      1:      2:      3:   test[6][9]={8, 3, 15}
   6:      7:      1:      2:      3:   test[

   6:      12:      13:      14:      8:      9:      10:   11->5
   6:      12:      13:      14:      8:      9:      10:      11:   test[6][5]={11, 4}
   6:      12:      13:      14:      8:      9:      10:      11:   test[8][5]={11, 4}
   6:      12:      13:      14:      8:      9:      10:      11:   test[9][5]={11, 4}
   6:      12:      13:      14:      8:      9:      10:      11:   test[10][5]={11, 4}
   6:      12:      13:      14:      8:      9:      10:      11:   test[11][5]={11}
   6:      12:      13:      14:      8:      9:      10:      11:   test[12][5]={11, 4}
   6:      12:      13:      14:      8:      9:      10:      11:   test[13][5]={11, 4}
   6:      12:      13:      14:      8:      9:      10:      11:   test[14][5]={11, 4}
   6:      12:      13:      14:      8:      9:      10:      11:   test[18][5]={11, 4}
   6:      12:      13:      14:      8:      9:      10:      11:   5; pred: {6, 8, 9, 10, 11, 12, 13, 14, 18}
   6:      12:      13:    

In [90]:
print(test[19])

[-1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0
 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0]


In [15]:
test = np.array(np.zeros(shape=(nb_nodes, nb_nodes))-1,dtype=object)
for o in graph_dict.keys():
    for d in graph_dict[o].keys():
        if test[o-1][d-1]==-1:
            test[o-1][d-1] = set()
        test[o-1][d-1].add(o-1)# += {o-1}
    

In [85]:
print(graph_dict[18][19])

KeyError: 19

In [105]:
print(test[18][6])

{0, 18, 12}


## Annex to compute the path

In [140]:
from collections import defaultdict 
   
#This class represents a directed graph  
# using adjacency list representation 
class Graph: 
   
    def __init__(self,vertices): 
        #No. of vertices 
        self.V= vertices  
          
        # default dictionary to store graph 
        self.graph = defaultdict(list)  
   
    # function to add an edge to graph 
    def addEdge(self,u,v): 
        self.graph[u].append(v) 
   
    def printAllPathsUtil(self, u, d, visited, path): 
        # Mark the current node as visited and store in path 
        visited[u]= True
        path.append(u) 
  
        # If current vertex is same as destination, then print 
        # current path[] 
        if u ==d: 
            # print(path)
            path_tmp = np.zeros(shape=(nb_links))
            for i in range(len(path)-1):
                path_tmp[graph_dict[path[i]][path[i+1]]] = 1
            if self.paths_m.shape[0]==0:
                self.paths_m = path_tmp.reshape((1, nb_links))
            else:
                self.paths_m = np.append(self.paths_m, path_tmp.reshape((1, nb_links)), axis=0)
        else: 
            # If current vertex is not destination 
            #Recur for all the vertices adjacent to this vertex 
            for i in self.graph[u]: 
                if visited[i]==False: 
                    self.printAllPathsUtil(i, d, visited, path) 
                      
        # Remove current vertex from path[] and mark it as unvisited 
        path.pop() 
        visited[u]= False
   
   
    # Prints all paths from 's' to 'd' 
    def printAllPaths(self,s, d): 
        self.paths_m = np.array([])
        # Mark all the vertices as not visited 
        visited =[False]*(self.V)
  
        # Create an array to store paths 
        path = [] 
        
        # Call the recursive helper function to print all paths 
        self.printAllPathsUtil(s, d,visited, path) 
        return self.paths_m 

# argument graph
def delta_matrix():
    route2od = np.array([])
    paths_matrix = np.array([])
    g = Graph(nb_links) 
    for line in graph:
        g.addEdge(int(line[1]), int(line[2]))

    for i in range(demand.shape[0]):
        s = int(demand[i][0]) ; d = int(demand[i][1])
        paths_matrix_tmp = g.printAllPaths(s, d)
        route2od_tmp = np.ones(paths_matrix_tmp.shape[0])
        route2od_tmp = route2od_tmp*i
        if paths_matrix.shape[0]==0:
            paths_matrix = paths_matrix_tmp
        else:
            paths_matrix = np.append(paths_matrix, paths_matrix_tmp, axis=0)
        route2od = np.append(route2od, route2od_tmp)
    return paths_matrix, route2od

In [141]:
paths_matrix, route2od = delta_matrix()

In [146]:
print(paths_matrix[:,15])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
